In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from ecgdetectors import Detectors
import matplotlib.pyplot as plt
import seaborn as sns
from time import time as time

In [2]:
%%time
# df = pd.read_csv("/hdd/physio/af/.csv", index_col = 0)
final = pd.read_csv('/hdd/physio/af/hugedfrpeak5sec.csv', index_col = 0)
final

CPU times: user 1min 43s, sys: 6.69 s, total: 1min 50s
Wall time: 2min


,af,0,1,2,3,4,5,6,7,8,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787506,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787507,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787508,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787509,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
fs = 250

def pipelinedRpeakExtraction(x, fs):
    detectors = Detectors(fs)
#     x = detectors.swt_detector(x)
#     x = detectors.hamilton_detector(x)
    x = detectors.pan_tompkins_detector(x)
    return x

def optimizedFeatureExtraction(x, fs):
    rr = np.diff(x) / fs 
    hr = 60 / rr
    rmssd = np.sqrt(np.mean(np.square(rr)))
    sdnn = np.std(rr)
    mean_rr = np.mean(rr)
    mean_hr = np.mean(hr)
    std_hr = np.std(hr)
    min_hr = np.min(hr)
    max_hr = np.max(hr)
    features = np.array([rmssd, sdnn, mean_rr, mean_hr, std_hr, min_hr, max_hr])
    return features

def helperMethod():
    return np.array(final.loc[5])[1:]


x = helperMethod()
print(x.shape)
start = time()
x = pipelinedRpeakExtraction(x, fs)
print(np.array(x).shape, x)
features = optimizedFeatureExtraction(x, fs)
print(features.shape)
print(time() - start)

(1250,)
(5,) [188, 422, 634, 851, 1053]
(7,)
0.019435644149780273


In [87]:
x = np.array(x)
x.shape

(5,)

In [94]:
chdb = pd.read_csv('/hdd/physio/af2/finaldfs/2017final_normal_ecg_L5_S1.csv')

In [95]:
chdb

,af,0,1,2,3,4,5,6,7,8,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249
0,0,-0.119671,-0.171568,-0.209467,-0.241545,-0.251298,-0.261781,-0.265077,-0.268898,-0.266928,...,-0.004762,-0.011451,-0.016941,-0.012288,-0.003720,-0.000231,0.002896,0.007219,0.010819,0.015448
1,0,0.179901,0.182646,0.186076,0.190405,0.194489,0.197974,0.201573,0.205480,0.210523,...,-0.099974,-0.101241,-0.102327,-0.103782,-0.102690,-0.098856,-0.094154,-0.085563,-0.073865,-0.057123
2,0,-0.080083,-0.073838,-0.066394,-0.056862,-0.043762,-0.025739,-0.003682,0.017315,0.036692,...,-0.062881,-0.066561,-0.068860,-0.073529,-0.078563,-0.089408,-0.105989,-0.113971,-0.117699,-0.121547
3,0,0.004971,0.009785,0.012551,0.013169,0.011561,0.008022,0.005519,0.001601,-0.000719,...,-0.027093,-0.030426,-0.032781,-0.035219,-0.037578,-0.039177,-0.041016,-0.041205,-0.043212,-0.044915
4,0,0.008039,0.006846,0.005272,0.002796,0.000349,-0.000125,-0.001080,-0.000778,-0.002517,...,-0.018165,-0.026139,-0.033500,-0.040133,-0.044640,-0.047706,-0.049499,-0.049575,-0.048598,-0.045659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163588,0,-0.026915,-0.021031,-0.014944,-0.009475,-0.004890,0.000104,0.006036,0.013603,0.019909,...,-0.008041,-0.009067,-0.008895,-0.009538,-0.010804,-0.012067,-0.014235,-0.015384,-0.016604,-0.017808
163589,0,0.273103,0.281178,0.288176,0.293863,0.300131,0.305001,0.310949,0.315261,0.318010,...,-0.041078,-0.040873,-0.041422,-0.042109,-0.041949,-0.042029,-0.041985,-0.042000,-0.042037,-0.041325
163590,0,0.083085,0.065010,0.046886,0.029653,0.010828,-0.013283,-0.039487,-0.053796,-0.064019,...,-0.059016,-0.057726,-0.056972,-0.056561,-0.055091,-0.054100,-0.052678,-0.051961,-0.052033,-0.051343
163591,0,0.004186,0.001340,0.000015,-0.000567,-0.001093,-0.001900,-0.003228,-0.004483,-0.004620,...,-0.105929,-0.100147,-0.095189,-0.088889,-0.082454,-0.074770,-0.068059,-0.060077,-0.052014,-0.042487


In [96]:
vf = pd.read_csv('/hdd/physio/vf2/finaldfs/final5sec.csv')
vf

,Unnamed: 0,vf,0,1,2,3,4,5,6,7,...,1240,1241,1242,1243,1244,1245,1246,1247,1248,1249
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18746,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18747,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
18749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2><ins>Pi deploy baseline:</ins></h2>

In [97]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from ecgdetectors import Detectors
import matplotlib.pyplot as plt
import seaborn as sns
from time import time as time

In [98]:
%%time

afd = pd.read_csv('/hdd/physio/af/hugedfrpeak5sec.csv', index_col = 0)
chd = pd.read_csv('/hdd/physio/af2/finaldfs/2017final_normal_ecg_L5_S1.csv')

CPU times: user 2min 26s, sys: 2min 2s, total: 4min 29s
Wall time: 6min 27s


In [ ]:
def unpackJoblib(model):
    times = []
    for i in range(int(sys.argv[1])):
        start = time.time()
        x = pipelinedRpeakExtraction(window, fs)
        features = optimizedFeatureExtraction(x, fs)
        pred = model.predict(features)
        end = time.time()
        times.append(end - start)

    print(np.mean(times))
    
def helperMethod():
    for file in os.listdir('AFDB/'):
        model = joblib.load(file)
        print('\n\n' + str(file)[5:7] + ":\n")
        unpackJoblib(model)
        print('---------------')
        
#     for file in os.listdir('CHDB/'):
#         model = joblib.load(file)
#         print('\n\n' + str(file)[5:7] + ":\n")
#         unpackJoblib(model)
#         print('---------------')
    

In [100]:
a = np.load('/hdd/physio/edgeml/examples/pytorch/Bonsai/traditional models/1window.npy')

0.0

# Top features to be deployed on Pi

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from ecgdetectors import Detectors
import matplotlib.pyplot as plt
import seaborn as sns
from time import time as time

In [ ]:
# Individual Features

def get_nni_20(nn_intervals, fs):
    

def get_mean_nni(nn_intervals, fs):
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    return np.mean(nn_intervals)

In [6]:
def _2017_top_4_features(nn_intervals, fs):
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    return nni_50, pnni_50, nni_20, cvsd

def _2017_top_6_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    max_hr = max(heart_rate_list)
    
    return nni_50, pnni_50, nni_20, cvsd, cvnni, max_hr

def _2017_top_8_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    max_hr = max(heart_rate_list)
    mean_hr = np.mean(heart_rate_list)
    sdsd = np.std(diff_nni)
    
    return nni_50, pnni_50, nni_20, cvsd, cvnni, max_hr, mean_hr, sdsd

def _2017_top_10_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    max_hr = max(heart_rate_list)
    mean_hr = np.mean(heart_rate_list)
    sdsd = np.std(diff_nni)
    
    std_hr = np.std(heart_rate_list)
    pnni_20 = 100 * nni_20 / length_int
    
    
    return nni_50, pnni_50, nni_20, cvsd, cvnni, max_hr, mean_hr, sdsd, std_hr, pnni_20

def _2017_top_12_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    max_hr = max(heart_rate_list)
    mean_hr = np.mean(heart_rate_list)
    sdsd = np.std(diff_nni)
    
    std_hr = np.std(heart_rate_list)
    pnni_20 = 100 * nni_20 / length_int
    
    rmssd = np.sqrt(np.mean(diff_nni ** 2))
    
    return nni_50, pnni_50, nni_20, cvsd, cvnni, max_hr, mean_hr, sdsd, std_hr, pnni_20, rmssd, sdnn

def _2017_top_14_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    max_hr = max(heart_rate_list)
    mean_hr = np.mean(heart_rate_list)
    sdnn = np.std(nn_intervals, ddof = 1)
    
    std_hr = np.std(heart_rate_list)
    pnni_20 = 100 * nni_20 / length_int
    
    rmssd = np.sqrt(np.mean(diff_nni ** 2))
    sdsd = np.std(diff_nni)    
    min_hr = min(heart_rate_list)
    mean_nni = np.mean(nn_intervals)
    
    return nni_50, pnni_50, nni_20, cvsd, cvnni, max_hr, mean_hr, sdsd, std_hr, pnni_20, rmssd, sdnn, min_hr, mean_nni
    

def afdb_top_4_features(nn_intervals, fs): 
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    pnni_20 = 100 * nni_20 / length_int
    
    return nni_20, nni_50, pnni_20, pnni_50
    
    
def afdb_top_6_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    pnni_20 = 100 * nni_20 / length_int
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    return nni_20, nni_50, pnni_20, pnni_50, cvnni, cvsd

def afdb_top_8_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    pnni_20 = 100 * nni_20 / length_int
    
    sdnn = np.std(nn_intervals, ddof = 1)
    
    cvnni = sdnn / np.mean(nn_intervals)
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    std_hr = np.std(heart_rate_list)
    return nni_20, nni_50, pnni_20, pnni_50, cvnni, cvsd, sdnn, std_hr 

def afdb_top_10_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    pnni_20 = 100 * nni_20 / length_int
    
    sdnn = np.std(nn_intervals, ddof = 1)

    cvnni = sdnn / np.mean(nn_intervals)
    cvsd = np.sqrt(np.mean(diff_nni ** 2)) / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    std_hr = np.std(heart_rate_list)
    
    max_hr = max(heart_rate_list)
    sdsd = np.std(diff_nni)
    
    return nni_20, nni_50, pnni_20, pnni_50, cvnni, cvsd, sdnn, std_hr, max_hr, sdsd

def afdb_top_12_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    pnni_20 = 100 * nni_20 / length_int
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    rmssd = np.sqrt(np.mean(diff_nni ** 2))
    cvsd = rmssd / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    std_hr = np.std(heart_rate_list)
    
    max_hr = max(heart_rate_list)
    sdsd = np.std(diff_nni)
    
    mean_hr = np.mean(heart_rate_list)
    
    
    return nni_20, nni_50, pnni_20, pnni_50, cvnni, cvsd, sdnn, std_hr, max_hr, sdsd, mean_hr, rmssd

def afdb_top_14_features(nn_intervals, fs):
    
    diff_nni = np.diff(nn_intervals)
    length_int = len(nn_intervals)
    
    nni_50 = sum(np.abs(diff_nni) > (50*fs/1000))
    pnni_50 = 100 * nni_50 / length_int
    
    nni_20 = sum(np.abs(diff_nni) > (20*fs/1000))
    pnni_20 = 100 * nni_20 / length_int
    
    sdnn = np.std(nn_intervals, ddof = 1)
    cvnni = sdnn / np.mean(nn_intervals)
    rmssd = np.sqrt(np.mean(diff_nni ** 2))
    cvsd = rmssd / np.mean(nn_intervals)
    
    heart_rate_list = np.divide(60000, nn_intervals)
    std_hr = np.std(heart_rate_list)
    
    max_hr = max(heart_rate_list)
    sdsd = np.std(diff_nni)
    
    mean_hr = np.mean(heart_rate_list)
    
    min_hr = min(heart_rate_list)
    mean_nni = np.mean(nn_intervals)
    
    return nni_20, nni_50, pnni_20, pnni_50, cvnni, cvsd, sdnn, std_hr, max_hr, sdsd, mean_hr, rmssd, min_hr, mean_nni

In [18]:
# sathyama therla

def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

normalize([1, 2])

array([0., 1.])

In [12]:
from scipy.signal import butter, lfilter,filtfilt, iirnotch
from scipy.signal import freqs
import matplotlib.pyplot as plt
from scipy.signal import medfilt

In [28]:
%%time
fs = 250
n = 0.5*fs
f_high = 0.5
cut_off = f_high/n

order = 4

data = np.arange(250)
data = normalize(data)

b,a = butter(order, cut_off,btype='high')
high_filtered_data = filtfilt(b, a, data)

CPU times: user 3.04 ms, sys: 0 ns, total: 3.04 ms
Wall time: 2.19 ms
